In [12]:
import torch
from datasets import load_dataset
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from google.colab import files

# Ensure GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU.")

print("\n--- Loading and Preprocessing Data ---")

# Upload the CSV from your computer
uploaded = files.upload()

# Replace with your actual filename if it's different
dataset = load_dataset("csv", data_files=list(uploaded.keys()))
print(dataset)

# Split into train/test
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_data = dataset["train"].select(range(min(2000, len(dataset["train"]))))
eval_data = dataset["test"].select(range(min(500, len(dataset["test"]))))

GPU not available, using CPU.

--- Loading and Preprocessing Data ---


Saving clickbait_data.csv to clickbait_data (5).csv


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['headline', 'clickbait'],
        num_rows: 32000
    })
})


In [18]:

# Initialize Tokenizer (the pre-processing tool)
MODEL_NAME = "distilbert-base-multilingual-cased"
tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    # This prepares the text and converts it into numerical IDs that the model understands
    return tokenizer(examples["headline"], truncation=True, padding=True)

# Apply tokenization to the training and evaluation sets
tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_eval = eval_data.map(tokenize_function, batched=True)

# Set the format to PyTorch tensors
tokenized_train.set_format("torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_eval.set_format("torch", columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [19]:
# --- 3. MODEL DEFINITION (Stage 3 Analog) ---

# Load the pre-trained DistilBERT model for sequence classification (sentiment analysis)
# The model automatically head for 2 classes (positive/negative)
model = DistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)
print(f"Model loaded: {MODEL_NAME}")

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded: distilbert-base-multilingual-cased


In [30]:
from google.colab import drive
drive.mount('/content/drive')
from transformers import TrainingArguments, Trainer
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

# --- Compute Metrics Function ---
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds, average="binary")
    return {"accuracy": acc, "f1": f1}

# --- Define Experiments (only 3 hyperparameters assigned to you) ---
EXPERIMENTS = [
    {"num_train_epochs": 2, "per_device_train_batch_size": 16, "weight_decay": 0.01},
    {"num_train_epochs": 4, "per_device_train_batch_size": 32, "weight_decay": 0.05},
    {"num_train_epochs": 6, "per_device_train_batch_size": 64, "weight_decay": 0.1},
]

results_list = []

for i, exp in enumerate(EXPERIMENTS, start=1):
    print(f"\n--- Running Experiment {i}: {exp} ---")

    training_args = TrainingArguments(
        output_dir=f"./results_exp_{i}",
        num_train_epochs=exp["num_train_epochs"],
        per_device_train_batch_size=exp["per_device_train_batch_size"],
        per_device_eval_batch_size=16,
        weight_decay=exp["weight_decay"],
        warmup_steps=500,
        logging_dir=f"./logs_exp_{i}",
        logging_steps=100,
        eval_strategy="epoch",     # <- your transformers version supports eval_strategy
        save_strategy="epoch",     # must match eval_strategy
        load_best_model_at_end=True,
        fp16=torch.cuda.is_available(),
        report_to=[],              # disable external logging like WandB
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    # Train & evaluate
    trainer.train()
    eval_results = trainer.evaluate()

    print(f"Experiment Results: {eval_results}")

    results_list.append({
        "num_train_epochs": exp["num_train_epochs"],
        "per_device_train_batch_size": exp["per_device_train_batch_size"],
        "weight_decay": exp["weight_decay"],
        "accuracy": eval_results.get("eval_accuracy"),
        "f1": eval_results.get("eval_f1")
    })

# --- Export Results to Excel ---
results_df = pd.DataFrame(results_list)
results_df.to_excel("/content/drive/MyDrive/experiment_results.xlsx", index=False)
print("\n✅ Results exported successfully to your Google Drive.")


Mounted at /content/drive

--- Running Experiment 1: {'num_train_epochs': 2, 'per_device_train_batch_size': 16, 'weight_decay': 0.01} ---


/tmp/ipython-input-3594931058.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.000200,0.016505,0.998000,0.998020
2,0.000000,0.022098,0.996000,0.996047


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Experiment Results: {'eval_loss': 0.016504881903529167, 'eval_accuracy': 0.998, 'eval_f1': 0.998019801980198, 'eval_runtime': 26.9196, 'eval_samples_per_second': 18.574, 'eval_steps_per_second': 1.189, 'epoch': 2.0}

--- Running Experiment 2: {'num_train_epochs': 4, 'per_device_train_batch_size': 32, 'weight_decay': 0.05} ---


...

Experiment Results: {'eval_loss': 0.06065290421247482, 'eval_accuracy': 0.992, 'eval_f1': 0.9921259842519685, 'eval_runtime': 26.7996, 'eval_samples_per_second': 18.657, 'eval_steps_per_second': 1.194, 'epoch': 4.0}

--- Running Experiment 3: {'num_train_epochs': 6, 'per_device_train_batch_size': 64, 'weight_decay': 0.1} ---


/tmp/ipython-input-3594931058.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.062303,0.992000,0.992126
2,No log,0.027209,0.996000,0.996047
3,No log,0.045205,0.996000,0.996047
4,0.000000,0.065956,0.994000,0.994083
5,0.000000,0.073061,0.994000,0.994083
6,0.000000,0.077176,0.994000,0.994083


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument i

Experiment Results: {'eval_loss': 0.027208801358938217, 'eval_accuracy': 0.996, 'eval_f1': 0.9960474308300395, 'eval_runtime': 25.8303, 'eval_samples_per_second': 19.357, 'eval_steps_per_second': 1.239, 'epoch': 6.0}

✅ Results exported successfully to your Google Drive.


In [31]:
from transformers import pipeline
import torch

# 1. Define the trained model and tokenizer (already from your training)
# If you restarted your runtime, you'll need to reload them from a checkpoint instead.
# For now, this assumes you still have `model` and `tokenizer` in memory.

# 2. Create a prediction pipeline using your fine-tuned model
clickbait_detector = pipeline(
    "text-classification",  # more general task name than 'sentiment-analysis'
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1  # GPU if available
)

# 3. Example new headlines to test
new_headlines = [
    "MMDA Launches New Traffic Scheme to Ease Congestion Along EDSA",
    "DepEd Confirms Opening of Classes Will Proceed as Scheduled",
    "DOH Reports Steady Decline in Dengue Cases Nationwide",
    "You Won’t Believe What This Filipino Celebrity Did After Winning the Lottery!",
    "This One Trick Can Help You Save Thousands on Your Meralco Bill!",
    "Students in Manila Tried This Study Method—The Results Will Shock You!",
    "Comelec Prepares for 2025 Elections With Improved Voter Registration System",
    "PH Economy Grows by 5.8% in Third Quarter, Says PSA",
    "A Mayor’s Secret Finally Revealed—The Whole Town Is Talking About It!",
    "Here’s Why Everyone Is Rushing to Try This New Food Trend in Quezon City!",
]

# 4. Run predictions
print("\n--- Running Clickbait Prediction ---")
results = clickbait_detector(new_headlines)

# 5. Print results
for text, result in zip(new_headlines, results):
    label = result["label"]
    # Depending on your dataset, 1 = clickbait, 0 = non-clickbait
    prediction = "Clickbait" if label in ["LABEL_1", "1"] else "Not Clickbait"
    print(f"Headline: {text}")
    print(f"   Prediction: {prediction} (Score: {result['score']:.4f})")

Device set to use cpu



--- Running Clickbait Prediction ---
Headline: MMDA Launches New Traffic Scheme to Ease Congestion Along EDSA
   Prediction: Not Clickbait (Score: 1.0000)
Headline: DepEd Confirms Opening of Classes Will Proceed as Scheduled
   Prediction: Not Clickbait (Score: 1.0000)
Headline: DOH Reports Steady Decline in Dengue Cases Nationwide
   Prediction: Not Clickbait (Score: 1.0000)
Headline: You Won’t Believe What This Filipino Celebrity Did After Winning the Lottery!
   Prediction: Clickbait (Score: 1.0000)
Headline: This One Trick Can Help You Save Thousands on Your Meralco Bill!
   Prediction: Clickbait (Score: 1.0000)
Headline: Students in Manila Tried This Study Method—The Results Will Shock You!
   Prediction: Clickbait (Score: 0.9999)
Headline: Comelec Prepares for 2025 Elections With Improved Voter Registration System
   Prediction: Not Clickbait (Score: 1.0000)
Headline: PH Economy Grows by 5.8% in Third Quarter, Says PSA
   Prediction: Not Clickbait (Score: 1.0000)
Headline: A May

In [32]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# 1. Get predictions from the trainer on your evaluation set
predictions = trainer.predict(tokenized_eval)

# 2. Convert logits to predicted labels
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

# 3. Generate evaluation report
print("\n--- Classification Report ---")
print(classification_report(y_true, y_pred, target_names=["Not Clickbait", "Clickbait"]))

# 4. Confusion matrix
print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_true, y_pred))


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



--- Classification Report ---
               precision    recall  f1-score   support

Not Clickbait       1.00      0.99      1.00       248
    Clickbait       0.99      1.00      1.00       252

     accuracy                           1.00       500
    macro avg       1.00      1.00      1.00       500
 weighted avg       1.00      1.00      1.00       500


--- Confusion Matrix ---
[[246   2]
 [  0 252]]
